In [ ]:
# python3 torch2tflite/converter.py \
# --torch-model-path chosen_models/mobilenetv2_model.pth \
# --tf-lite-model-path tflite_models/mobilenetv2.tflite \
# --test-im-path samples/test.jpg \
# --show-results \
# --convert

In [ ]:
# python3 torch2tflite/converter.py \
# --torch-model-path chosen_models/resnet18_model.pth \
# --tf-lite-model-path tflite_models/resnet18.tflite \
# --test-im-path samples/test.jpg \
# --show-results \
# --convert

In [ ]:
# cp /datasets/affwild2_latest/cropped_aligned_images/144/07804.jpg /scratch/users/nnayal17/moody_much/MoodyMuch/ml/samples/

In [2]:
import os
os.chdir("../")

In [3]:
import torch
import torch.nn as nn
from models.mobilenetv2 import mobilenetv2
from torchvision.models import resnet18

In [4]:
resnet_model = resnet18(num_classes=2)
resnet_model.load_state_dict(torch.load("chosen_models/resnet18.pth"))

<All keys matched successfully>

In [15]:
d = torch.load("chosen_models/resnet18.pth")

In [4]:
mobilenetv2_model = mobilenetv2()
mobilenetv2_model.classifier = nn.Linear(mobilenetv2_model.classifier.in_features, 2)
mobilenetv2_model.load_state_dict(torch.load("chosen_models/mobilenetv2.pth"))

<All keys matched successfully>

In [5]:
example = torch.rand(1, 3, 112, 112)

In [6]:
traced_script_module = torch.jit.trace(mobilenetv2_model, example)

In [7]:
traced_script_module.save("pytorch_mobile/mobilenetv2.pt")

In [ ]:
traced_script_module = torch.jit.trace(resnet_model, example)
traced_script_module.save("pytorch_model/resnet18.pt")

# Hand-Coded Method

In [32]:
import onnx
import torch
import numpy as np
import matplotlib.image as mpimg

In [33]:
def get_example_input():
    img = mpimg.imread("samples/test.jpg")    
    return torch.from_numpy(img.astype(np.float32)).unsqueeze(0).permute(0, 3, 1, 2)

def get_pytorch_model():
    model = torch.load("chosen_models/mobilenetv2_model.pth")
    return model

In [34]:
example_input = get_example_input() # exmample for the forward pass input 
pytorch_model = get_pytorch_model()
ONNX_PATH="onnx/mobilenetv2.onnx"

In [35]:


torch.onnx.export(
    model=pytorch_model,
    args=example_input, 
    f=ONNX_PATH, # where should it be saved
    verbose=False,
    export_params=True,
    do_constant_folding=False,  # fold constant values for optimization
    # do_constant_folding=True,   # fold constant values for optimization
    input_names=['input'],
    output_names=['output']
)
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

In [43]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "tflite/mobilenetv2.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "onnx/mobilenetv2.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

INFO:tensorflow:Assets written to: tflite/mobilenetv2.pb/assets


INFO:tensorflow:Assets written to: tflite/mobilenetv2.pb/assets


In [39]:
import tensorflow as tf

In [45]:
TF_PATH_T = "tflite/mobilenetv2.pb/saved_model.pb" 
TFLITE_PATH = "tflite_models/mobilenetv2.tflite"
# protopuf needs your virtual environment to be explictly exported in the path
#os.environ["PATH"] = "/opt/miniconda3/envs/convert/bin:/opt/miniconda3/bin:/usr/local/sbin:...."

# make a converter object from the saved tensorflow file
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(TF_PATH_T,  # TensorFlow freezegraph .pb model file
                                                      input_arrays=['input'], # name of input arrays as defined in torch.onnx.export function before.
                                                      output_arrays=['output'] # name of output arrays defined in torch.onnx.export function before.
                                                      )

# tell converter which type of optimization techniques to use
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]

converter.experimental_new_converter = True

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

Ignore 'tcmalloc: large alloc' warnings.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 8: invalid start byte

In [37]:
!which pip

/kuacc/users/nnayal17/.conda/envs/nazir_env/bin/pip


In [ ]:
# export PATH=$PATH:/kuacc/users/nnayal17/.conda/envs/nazir_env/tensorflow/bin

# Pytorch To Keras


In [8]:
from pytorch2keras import pytorch_to_keras
from torch.autograd import Variable
import numpy as np

In [19]:
mobilenetv2_model = mobilenetv2()
mobilenetv2_model.classifier = nn.Linear(mobilenetv2_model.classifier.in_features, 2)
mobilenetv2_model.load_state_dict(torch.load("chosen_models/mobilenetv2.pth"))
mobilenetv2_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
       

In [9]:
input_np = np.random.uniform(0, 1, (1, 3, 112, 112))
input_var = Variable(torch.FloatTensor(input_np))

In [20]:
k_model = pytorch_to_keras(mobilenetv2_model, input_var, [(3, 112, 112)], verbose=True)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(3, 112, 112)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight 475 with shape (32, 3, 3, 3).
DEBUG:onnx2keras:Found weight 476 with shape (32,).
DEBUG:onnx2keras:Found weight 478 with shape (32, 1, 3, 3).
DEBUG:onnx2keras:Found weight 479 with shape (32,).
DEBUG:onnx2keras:Found weight 481 with shape (16, 32, 1, 1).
DEBUG:onnx2keras:Found weight 482 with shape (16,).
DEBUG:onnx2keras:Found weight 484 with shape (96, 16, 1, 1).
DEBUG:onnx2keras:Found weight 485 with shape (96,).
DEBUG:onnx2keras:Found weight 487 with shape (96, 1, 3, 3).
DEBUG:onnx2keras:Found we

DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 32, 56, 56), dtype=tf.float32, name=None), name='317/Relu6:0', description="created by layer '317'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 477
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 32, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 317).
DEBUG:onnx2keras:Check input 1 (name 478).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 479).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy 

graph(%input_0 : Float(1:37632, 3:12544, 112:112, 112:1, requires_grad=0, device=cpu),
      %classifier.weight : Float(2:1280, 1280:1, requires_grad=1, device=cpu),
      %classifier.bias : Float(2:1, requires_grad=1, device=cpu),
      %475 : Float(32:27, 3:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %476 : Float(32:1, requires_grad=0, device=cpu),
      %478 : Float(32:9, 1:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %479 : Float(32:1, requires_grad=0, device=cpu),
      %481 : Float(16:32, 32:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %482 : Float(16:1, requires_grad=0, device=cpu),
      %484 : Float(96:16, 16:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %485 : Float(96:1, requires_grad=0, device=cpu),
      %487 : Float(96:9, 1:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %488 : Float(96:1, requires_grad=0, device=cpu),
      %490 : Float(24:96, 96:1, 1:1, 1:1, requires_grad=0, device=cpu),
      %491 : Float(24:1, requires_grad=0, device=cpu),
      %493 

DEBUG:onnx2keras:node_name: 492
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 489).
DEBUG:onnx2keras:Check input 1 (name 493).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 494).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 144, 28, 28), dtype=tf.float32, name=None), name='492/BiasAdd:0', description="created by layer '492'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:..

DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 345
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 504).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 144, 14, 14), dtype=tf.float32, name=None), name='345/Relu6:0', description="created by layer '345'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 507
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEB

...

DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 579
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 414).
DEBUG:onnx2keras:Check input 1 (name 580).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 581).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 96, 7, 7), dtype=tf.float32, na

DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 429
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 591).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:clip:Using ReLU(6.0) instead of clip
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 576, 7, 7), dtype=tf.float32, name=None), name='429/Relu6:0', description="created by layer '429'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 594
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 576, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [2, 2], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 429).
DEB

DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Add
DEBUG:onnx2keras:node_name: 443
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 597).
DEBUG:onnx2keras:Check input 1 (name 606).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:add:Convert inputs to Keras/TF layers if needed.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 160, 4, 4), dtype=tf.float32, name=None), name='443/add:0', description="created by layer '443'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 609
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [1, 1], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all

DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 623).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:conv:Number of groups is equal to input channels, use DepthWise convolution
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 960, 4, 4), dtype=tf.float32, name=None), name='621/BiasAdd:0', description="created by layer '621'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: 458
DEBUG:onnx2keras:node_params: {'max': 6.0, 'min': 0.0, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:

DEBUG:onnx2keras:type: Reshape
DEBUG:onnx2keras:node_name: 472
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 464).
DEBUG:onnx2keras:Check input 1 (name 471).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:reshape:The second argument is numpy array.
DEBUG:onnx2keras:reshape:The first argument is Keras/tf layer. Apply keras.Reshape.
DEBUG:onnx2keras:reshape:Target shape :
DEBUG:onnx2keras:reshape:[-1]
DEBUG:onnx2keras:reshape:The first argument is Keras/tf layer. Apply keras.Flatten.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 1280), dtype=tf.float32, name=None), name='472/Reshape:0', description="created by layer '472'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Gemm
DEBUG:onnx2keras:node_name: output_0
DEBUG:onnx2keras:node_params

In [13]:
k_model

tensorflow.python.keras.engine.functional.Functional

In [14]:
import tensorflow as tf

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(k_model)
tflite_model = converter.convert()

# Save the model.
with open('tflite_models/mobilenetv2.tflite', 'wb') as f:
    f.write(tflite_model)

/kuacc/users/nnayal17/.conda/envs/nazir_env/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: /tmp/tmp0508gec6/assets


INFO:tensorflow:Assets written to: /tmp/tmp0508gec6/assets


In [22]:
resnet_model = resnet18(num_classes=2)
resnet_model.load_state_dict(torch.load("chosen_models/resnet18.pth"))
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
k_model = pytorch_to_keras(resnet_model, input_var, [(3, 112, 112)], verbose=True)

k_model

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(k_model)
tflite_model = converter.convert()

# Save the model.
with open('tflite_models/resnet18.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(3, 112, 112)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight 193 with shape (64, 3, 7, 7).
DEBUG:onnx2keras:Found weight 194 with shape (64,).
DEBUG:onnx2keras:Found weight 196 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 197 with shape (64,).
DEBUG:onnx2keras:Found weight 199 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 200 with shape (64,).
DEBUG:onnx2keras:Found weight 202 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found weight 203 with shape (64,).
DEBUG:onnx2keras:Found weight 205 with shape (64, 64, 3, 3).
DEBUG:onnx2keras:Found 

graph(%input_0 : Float(1:37632, 3:12544, 112:112, 112:1, requires_grad=0, device=cpu),
      %fc.weight : Float(2:512, 512:1, requires_grad=1, device=cpu),
      %fc.bias : Float(2:1, requires_grad=1, device=cpu),
      %193 : Float(64:147, 3:49, 7:7, 7:1, requires_grad=0, device=cpu),
      %194 : Float(64:1, requires_grad=0, device=cpu),
      %196 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %197 : Float(64:1, requires_grad=0, device=cpu),
      %199 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %200 : Float(64:1, requires_grad=0, device=cpu),
      %202 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %203 : Float(64:1, requires_grad=0, device=cpu),
      %205 : Float(64:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %206 : Float(64:1, requires_grad=0, device=cpu),
      %208 : Float(128:576, 64:9, 3:3, 3:1, requires_grad=0, device=cpu),
      %209 : Float(128:1, requires_grad=0, device=cpu),
      %211 : Flo

DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 64, 56, 56), dtype=tf.float32, name=None), name='192/BiasAdd:0', description="created by layer '192'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 125
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 192).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 64, 56, 56), dtype=tf.float32, name=None), name='125/Relu:0', description="created by layer '125'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MaxPool
DEBUG:onnx2keras:node_name: 126
DEBUG:onnx2keras:node_params: {'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [2, 2], 'ch

DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Add
DEBUG:onnx2keras:node_name: 139
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 204).
DEBUG:onnx2keras:Check input 1 (name 133).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:add:Convert inputs to Keras/TF layers if needed.
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 64, 28, 28), dtype=tf.float32, name=None), name='139/add:0', description="created by layer '139'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 140
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 

DEBUG:onnx2keras:node_name: 219
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 152).
DEBUG:onnx2keras:Check input 1 (name 220).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 221).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 128, 14, 14), dtype=tf.float32, name=None), name='219/BiasAdd:0', description="created 

DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 256, 7, 7), dtype=tf.float32, name=None), name='231/BiasAdd:0', description="created by layer '231'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Relu
DEBUG:onnx2keras:node_name: 168
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 231).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 256, 7, 7), dtype=tf.float32, name=None), name='168/Relu:0', description="created by layer '168'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DE

DEBUG:onnx2keras:Output TF Layer -> KerasTensor(type_spec=TensorSpec(shape=(None, 512, 4, 4), dtype=tf.float32, name=None), name='181/Relu:0', description="created by layer '181'")
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 246
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [3, 3], 'pads': [1, 1, 1, 1], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 181).
DEBUG:onnx2keras:Check input 1 (name 247).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name 248).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:

INFO:tensorflow:Assets written to: /tmp/tmpeb99d5z2/assets


/kuacc/users/nnayal17/.conda/envs/nazir_env/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
INFO:tensorflow:Assets written to: /tmp/tmpeb99d5z2/assets
